In [38]:
"""Binary Exponetial Backoff Algorithm"""
import random
import simpy
import math

RANDOM_SEED = 29
SIM_TIME = 1000000
MU = 1
TS = 1 #time slot
N_HOST = 10 #number of hosts

""" Host class """
class Host:
    def __init__(self, env, arrival_rate):
        self.env = env
        self.arrival_rate = arrival_rate
        self.L = 0
        self.S = 0
        #self.arrives = 0
        self.N = 0 #times retransmitted
        self.server = simpy.Resource(env, capacity = 1)
        env.process(self.packets_arrival(env))
        
    def packets_arrival(self, env):
        while True:
            yield env.timeout(random.expovariate(self.arrival_rate))
            #self.arrives += 1
            if (self.L == 0): #no packet in the queue
                self.S = math.floor(env.now) + 1
                self.N = 0
            self.L += 1
        
    def process_packet(self, env):
        self.L -= 1
        if (self.L > 0):
            self.N = 0
            self.S = math.floor(env.now) + 1
    
    def exp_backoff(self, env):
        r = min(self.N, N_HOST)
        self.S = math.floor(env.now + random.randint(0,2**r)*TS) + 1
        self.N += 1
        
    def linear_backoff(self, env):
        K = min(self.N, 1024)
        self.S = math.floor(env.now + random.randint(0, K)*TS) + 1
        self.N += 1

""" Queue system  """		
class Ethernet:
    def __init__(self, env, arrival_rate, Type):
        self.env = env
        self.hosts = [Host(env, arrival_rate) for i in range(N_HOST)]
        self.success = 0
        self.collide = 0
        self.blank = 0
        self.Type = Type 
        self.arrival_rate = arrival_rate

    def sim(self, env):
        while True:
            hostLst = []
            for host in self.hosts:
                if (host.L > 0 and (host.S == math.floor(env.now))):#host is requesting now
                    hostLst.append(host)
            #print(len(hostLst))
            if (len(hostLst) == 1):#succeed
                hostLst[0].process_packet(env)
                self.success += 1
            elif (len(hostLst) > 1):#collide
                self.collide += 1
                for host in hostLst:
                    if (self.Type == 0):#exponetial
                        host.exp_backoff(env)
                    else:
                        host.linear_backoff(env)
            else:
                self.blank += 1
            yield self.env.timeout(TS)
                

def main():
	print("Ethernet system model:mu = {0}, Ts = {1}, N = {2}".format(MU, TS, N_HOST))

	random.seed(RANDOM_SEED)
	for Type in [0,1]:
		if (Type == 0):
			print ("For Exponetial Backoff Algorithm")
		else:
			print ("For Linear Backoff Algorithm")
		print ("{0:<9} {1:<9} {2:<9} {3:<9} {4:<9}".format(
			"Lambda", "Success", "Failed","Blank", "Throughput"))
		for arrival_rate in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]:
			env = simpy.Environment()
			ethernet = Ethernet(env, arrival_rate, Type)
			env.process(ethernet.sim(env))
			env.run(until=SIM_TIME)
			#for i in range(10):
				#print(ethernet.hosts[i].arrives)
			#print(total)
			print ("{0:<9.3f} {1:<9} {2:<9}{3:<9} {4:<9.5f}".format(
				arrival_rate,
				ethernet.success,
				ethernet.collide,
				ethernet.blank,
				ethernet.success/SIM_TIME))
	
if __name__ == '__main__': main()

Ethernet system model:mu = 1, Ts = 1, N = 10
For Exponetial Backoff Algorithm
Lambda    Success   Failed    Blank     Throughput
0.010     99876     9259     890865    0.09988  
0.020     200186    47015    752799    0.20019  
0.030     300151    137241   562608    0.30015  
0.040     399742    193964   406294    0.39974  
0.050     499488    172628   327884    0.49949  
0.060     598620    131757   269623    0.59862  
0.070     698978    93424    207598    0.69898  
0.080     796404    61827    141769    0.79640  
0.090     892039    36764    71197     0.89204  
For Linear Backoff Algorithm
Lambda    Success   Failed    Blank     Throughput
0.010     99829     15284    884887    0.09983  
0.020     200483    85326    714191    0.20048  
0.030     293187    592115   114698    0.29319  
0.040     289192    633604   77204     0.28919  
0.050     290455    632573   76972     0.29046  
0.060     289794    633053   77153     0.28979  
0.070     291135    632170   76695     0.29113  
0.080  